In [1]:
from __future__ import division
import numpy as np

In [2]:
from scipy.constants import epsilon_0, hbar, k as k_B, pi
from scipy.special import ellipk

All of these values are from Heather's memo on the nine-element array: http://cosmology.phys.columbia.edu/skip_wiki/lib/exe/fetch.php?media=meetings:2013-07-30_detectors:skip_9elementarray.pdf

In [3]:
t = 20e-9
f_r = 1e6 * np.array([100, 105, 110, 140, 145, 150, 180, 185, 190])
L_kinetic = 1.2e-12 # in H / square
# Heather says this is the sum of the geometric inductance from Sonnet and the assumed kinetic inductance from above.
L_total = 1e-9 * np.array([88.730, 89.027, 89.324, 91.108, 91.405, 91.702, 93.486, 93.783, 94.081])
A_C = 1e-6 * np.array([10.635, 7.692, 5.239, 5.285, 3.928, 2.744, 3.116, 2.352, 1.667])
w_C = 1e-6 * np.array([10, 8, 6, 10, 8, 6, 10, 8, 6])
g_C = w_C
p_C = w_C + g_C
C = 1e-12 * np.array([28.548, 25.807, 23.436, 14.185, 13.181, 12.277, 8.363, 7.892, 7.458])

Calculate the expected kinetic surface inductance, assuming $\Delta = 1.76 \, k_B \, (1.2 \, \mathrm{K})$ and the bulk resistivity $\rho_n = 24.5 \, \Omega \, \mathrm{nm}$:

In [4]:
1e12 * (hbar * 24.5 / 20) / (pi * 1.76 * k_B * 1.2) # in pH / square

1.4102149821529861

Why doesn't this match the above? Try using the measured $T_c$:

In [5]:
1e12 * (hbar * 24.5 / 20) / (pi * 1.76 * k_B * 1.46) # in pH / square

1.1590808072490297

These values are taken from a nine-element mask file:

In [6]:
w_L = 2e-6 # The inductor trace width.
g_L = 123e-6 # The inductor trace gap.
p_L = w_L + g_L # This is the measured pitch between trace centers.
N_tines = 16 # The number of inductor tines, ignoring the outer trace and perpendicular trace.
X_L = 3e-3
Y_L = 2e-3

Calculate the number of squares in the inductor using the formula and dimentions from the paper:

In [7]:
N_squares_paper = X_L * Y_L / (w_L * p_L)
N_squares_paper

24000.0

Calculate the number of squares in the inductor using measurements from the nine-element mask file. The traces run back and forth in the direction of $X_L$. Count corner squares in this direction. Stop at the beginning of the wider capacitor trace.

In [8]:
L_L = (N_tines * X_L + (X_L + 200e-6) +
       (Y_L + 42e-6) + 8 * g_L + 42e-6)
1e3 * L_L # in mm

54.268

In [9]:
N_squares = L_L / w_L
N_squares

27134.000000000004

The formula underestimates a little.

Back out the geometric inductance from Sonnet, assuming the formula from the paper was used:

In [10]:
L_g = L_total - N_squares_paper * L_kinetic
1e9 * L_g # in nH

array([ 59.93 ,  60.227,  60.524,  62.308,  62.605,  62.902,  64.686,
        64.983,  65.281])

Calculate the inductor volume, assuming a pure aluminum thickness of 20 nm and negligible oxide thickness:

In [11]:
V_L = L_L * w_L * t
1e18 * V_L # in cubic microns

2170.72

Calculate the expected capacitance using Jonas's equation 3 for an interdigitated capacitor:
\begin{equation*}
C
  =
  \epsilon_0 (1 + \epsilon_r) \frac{A_C}{p_C} \frac{K}{K'},
\end{equation*}
where variables are as defined in the memo.

In [12]:
epsilon_r = 11.9
k = np.tan(pi * w_C / (4 * p_C))**2
# For equal gap width and spacing, K / K' = 1/2.
K = ellipk(k**2)
Kp = ellipk(1 - k**2)
C_calc = epsilon_0 * (1 + epsilon_r) * (A_C / p_C) * (K / Kp)

In [13]:
C_calc / C

array([ 1.06375167,  1.06387405,  1.06387869,  1.06388357,  1.06368144,
        1.06369973,  1.06393183,  1.06374787,  1.06375207])

These calculated capacitances are about 6% larger than nominal.

Calculate the expected kinetic inductance fraction, assuming the nominal values above:

In [14]:
N_squares_paper * L_kinetic / L_total

array([ 0.32458019,  0.32349737,  0.32242175,  0.31610835,  0.31508123,
        0.31406076,  0.30806752,  0.3070919 ,  0.3061192 ])

The measured surface resistance in the normal state, just above $T_c$:

In [15]:
R_s = 4

The measured $T_c$:

In [16]:
T_c = 1.46

Assume BCS:

In [17]:
Delta = 1.76 * k_B * T_c

The expected kinetic surface inductance:

In [18]:
L_s = (hbar * R_s) / (pi * Delta)
1e12 * L_s # in pH / square

3.7847536563233626

The total predicted kinetic inductance:

In [19]:
L_k = N_squares * L_s
1e9 * L_k # in nH

102.69550571067812

Calculate the expected resonance frequencies using our best known values:

In [20]:
(2 * pi * ((L_k + L_g) * C_calc)**(1/2))**-1 / 1e6

array([  71.6172812 ,   75.25159724,   78.89442293,  100.85834223,
        104.54495257,  108.22748921,  130.41527078,  134.14316358,
        137.86831286])

These are somewhat too low.

In [21]:
L_k / (L_k + L_g)

array([ 0.63148462,  0.63033345,  0.62918648,  0.62238378,  0.62126553,
        0.62015128,  0.61354153,  0.6124548 ,  0.61136827])

This doesn't match the kinetic inductance fraction from the fits to Mattis-Bardeen plus TLS.

The situation still doesn't make sense. I'm not sure I believe the resistivity measurements.

The YBCO devices may have almost no kinetic inductance. If so, these would be the expected resonances:

In [22]:
sapphire_epsilon_r = 10  # approximately
k = np.tan(pi * w_C / (4 * p_C))**2
# For equal gap width and spacing, K / K' = 1/2.
K = ellipk(k**2)
Kp = ellipk(1 - k**2)
C_calc = epsilon_0 * (1 + sapphire_epsilon_r) * (A_C / p_C) * (K / Kp)

In [23]:
f_YBCO = (2 * pi * (L_g * C_calc)**(1/2))**-1
1e-6 * f_YBCO

array([ 127.75812627,  134.03220427,  140.30302487,  177.7399355 ,
        183.96455898,  190.16507726,  227.18285183,  233.34896337,
        239.49356331])